Import necessary libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import sagemaker
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/arshad/.config/sagemaker/config.yaml


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Load the employee data

In [80]:
data = pd.read_csv('employee_data.csv')
data

,ID,Gender,Experience (Years),Position,Salary
0,1,F,4,DevOps Engineer,109976
1,2,M,6,DevOps Engineer,120088
2,3,M,17,Web Developer,181301
3,4,M,7,Systems Administrator,77530
4,5,F,13,Systems Administrator,152397
...,...,...,...,...,...
395,396,F,19,Cloud Solutions Architect,236045
396,397,F,20,Web Developer,182770
397,398,F,9,Network Administrator,85550
398,399,M,18,Database Administrator (DBA),129996


In [81]:
# Rename the column
data.rename(columns={'Experience (Years)': 'Experience'}, inplace=True)

In [82]:
data["Gender"]=data["Gender"].astype('string')
data["Position"]=data["Position"].astype('string')
data.dtypes


ID                     int64
Gender        string[python]
Experience             int64
Position      string[python]
Salary                 int64
dtype: object

In [5]:
#dropping ID column
# data.drop(columns=['ID'], inplace=True)

Split the data into training and test data sets (70% split)

In [6]:
# train_data,test_data = train_test_split(data, test_size=0.3, random_state=42)

# # Display the number of rows in each dataset
# print(f'Total rows: {len(data)}')
# print(f'Training rows: {len(train_data)}')
# print(f'Testing rows: {len(test_data)}')

# # Save the split datasets into new CSV files
# train_data.to_csv('train_employee_data.csv', index=False)
# test_data.to_csv('test_employee_data.csv', index=False)

In [83]:
# Apply one-hot encoding
data_encoded = pd.get_dummies(data,drop_first=True)

# Display the encoded DataFrame
data_encoded = data_encoded.astype(int)

print(data_encoded.shape)
print(data_encoded.columns)

(400, 14)
Index(['ID', 'Experience', 'Salary', 'Gender_M',
       'Position_Database Administrator (DBA)', 'Position_DevOps Engineer',
       'Position_IT Manager', 'Position_IT Security Analyst',
       'Position_IT Support Specialist', 'Position_Network Administrator',
       'Position_Software Engineer', 'Position_Systems Administrator',
       'Position_Systems Analyst', 'Position_Web Developer'],
      dtype='object')


In [99]:
X=data_encoded.drop(columns="Salary")
X_columns=X.columns
Y=data_encoded["Salary"]
# Combine the feature column names with the target column name
columns_names = list(X.columns) + ["Salary"]

print(columns_names)



['ID', 'Experience', 'Gender_M', 'Position_Database Administrator (DBA)', 'Position_DevOps Engineer', 'Position_IT Manager', 'Position_IT Security Analyst', 'Position_IT Support Specialist', 'Position_Network Administrator', 'Position_Software Engineer', 'Position_Systems Administrator', 'Position_Systems Analyst', 'Position_Web Developer', 'Salary']


In [85]:
X

,ID,Experience,Gender_M,Position_Database Administrator (DBA),Position_DevOps Engineer,Position_IT Manager,Position_IT Security Analyst,Position_IT Support Specialist,Position_Network Administrator,Position_Software Engineer,Position_Systems Administrator,Position_Systems Analyst,Position_Web Developer
0,1,4,0,0,1,0,0,0,0,0,0,0,0
1,2,6,1,0,1,0,0,0,0,0,0,0,0
2,3,17,1,0,0,0,0,0,0,0,0,0,1
3,4,7,1,0,0,0,0,0,0,0,1,0,0
4,5,13,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,396,19,0,0,0,0,0,0,0,0,0,0,0
396,397,20,0,0,0,0,0,0,0,0,0,0,1
397,398,9,0,0,0,0,0,0,1,0,0,0,0
398,399,18,1,1,0,0,0,0,0,0,0,0,0


In [86]:
#Splitting dataset in training and testing data

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=42)

In [87]:
#Shape of testing and trainig data

X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((280, 13), (280,), (120, 13), (120,))

In [90]:
trainX = pd.DataFrame(X_train)
trainX['Salary'] = Y_train

testX = pd.DataFrame(X_test)
testX['Salary'] = Y_test

In [92]:
testX

,ID,Experience,Gender_M,Position_Database Administrator (DBA),Position_DevOps Engineer,Position_IT Manager,Position_IT Security Analyst,Position_IT Support Specialist,Position_Network Administrator,Position_Software Engineer,Position_Systems Administrator,Position_Systems Analyst,Position_Web Developer,Salary
209,210,11,1,1,0,0,0,0,0,0,0,0,0,93165
280,281,5,1,0,0,0,0,0,1,0,0,0,0,91842
33,34,3,1,0,0,0,1,0,0,0,0,0,0,97240
210,211,4,0,0,0,0,0,0,0,0,0,0,1,68000
93,94,16,1,0,0,0,1,0,0,0,0,0,0,128620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,61,7,1,1,0,0,0,0,0,0,0,0,0,105182
79,80,7,0,0,0,0,0,0,0,0,0,0,1,77058
285,286,11,1,0,1,0,0,0,0,0,0,0,0,180261
305,306,6,0,0,0,0,0,0,0,1,0,0,0,136900


In [ ]:


# Specify your S3 bucket and model file
model_file_path = 'model.joblib'
s3_model_path = 'data/model.joblib'  # The path where you want to store the model in S3

# Initialize the S3 client
s3 = boto3.client('s3')

# Upload the model to S3
s3.upload_file(model_file_path, bucket, s3_model_path)

print(f"Model uploaded to s3://{bucket}/{s3_model_path}")


Model uploaded to s3://mlops-salary-predictor-app/data/model.joblib


In [104]:
# Convert the NumPy array to a DataFrame
X_train_df=pd.DataFrame(trainX,columns=columns_names)
X_train_df
# Convert the NumPy array to a DataFrame
X_test_df=pd.DataFrame(testX,columns=columns_names)
X_test_df
Y_train_df=pd.DataFrame(Y_train)
Y_train_df
Y_test_df=pd.DataFrame(Y_train)
Y_test_df

,Salary
157,69668
109,123370
17,188681
347,242819
24,71211
...,...
71,110321
106,135066
270,115263
348,115769


In [105]:
# Save the split datasets into new CSV files
X_test_df.to_csv('X_test_df.csv', index=False)
X_train_df.to_csv('X_train_df.csv', index=False)
Y_test_df.to_csv('Y_test_df.csv', index=False)
Y_train_df.to_csv('Y_train_df.csv', index=False)

In [106]:
#Standardising features for similar scale

scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)

X_test=scaler.transform(X_test)


print(X_train)
print("\n")
print(f"shape of trainig data is {X_train.shape}")
print("\n")
print('*'*80)
print("\n")
print(X_test)
print("\n")
print(f"shape of testing data is {X_test.shape}")

[[-0.41085418 -0.32005212  0.98581488 ... -0.31994094 -0.29915575
  -0.34641016]
 [-0.84023834  0.65401954  0.98581488 ... -0.31994094 -0.29915575
  -0.34641016]
 [-1.66322463  0.97871009 -1.01438923 ... -0.31994094 -0.29915575
  -0.34641016]
 ...
 [ 0.59998768 -0.48239739 -1.01438923 ... -0.31994094 -0.29915575
  -0.34641016]
 [ 1.29773693 -0.15770684  0.98581488 ...  3.12557687 -0.29915575
  -0.34641016]
 [-0.90285686 -1.1317785  -1.01438923 ... -0.31994094 -0.29915575
  -0.34641016]]


shape of trainig data is (280, 13)


********************************************************************************


[[ 0.05431198  0.16698371  0.98581488 ... -0.31994094 -0.29915575
  -0.34641016]
 [ 0.68944271 -0.80708794  0.98581488 ... -0.31994094 -0.29915575
  -0.34641016]
 [-1.52009658 -1.1317785   0.98581488 ... -0.31994094 -0.29915575
  -0.34641016]
 ...
 [ 0.73417023  0.16698371  0.98581488 ... -0.31994094 -0.29915575
  -0.34641016]
 [ 0.91308029 -0.64474267 -1.01438923 ... -0.31994094 -0.

In [107]:
#Multiple linear regression
model=LinearRegression()

#Fitting the model
model.fit(X_train,Y_train)

#Prediciting bike rental duration
y_pred=model.predict(X_test)

In [108]:
X_test_df

,ID,Experience,Gender_M,Position_Database Administrator (DBA),Position_DevOps Engineer,Position_IT Manager,Position_IT Security Analyst,Position_IT Support Specialist,Position_Network Administrator,Position_Software Engineer,Position_Systems Administrator,Position_Systems Analyst,Position_Web Developer,Salary
209,210,11,1,1,0,0,0,0,0,0,0,0,0,93165
280,281,5,1,0,0,0,0,0,1,0,0,0,0,91842
33,34,3,1,0,0,0,1,0,0,0,0,0,0,97240
210,211,4,0,0,0,0,0,0,0,0,0,0,1,68000
93,94,16,1,0,0,0,1,0,0,0,0,0,0,128620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,61,7,1,1,0,0,0,0,0,0,0,0,0,105182
79,80,7,0,0,0,0,0,0,0,0,0,0,1,77058
285,286,11,1,0,1,0,0,0,0,0,0,0,0,180261
305,306,6,0,0,0,0,0,0,0,1,0,0,0,136900


In [109]:
MAE=mean_absolute_error(Y_test, y_pred)
print(f'Mean Absolute Error of model is {round(MAE,2)}')

Mean Absolute Error of model is 21599.67


In [45]:
X

,ID,Experience,Gender_M,Position_Database Administrator (DBA),Position_DevOps Engineer,Position_IT Manager,Position_IT Security Analyst,Position_IT Support Specialist,Position_Network Administrator,Position_Software Engineer,Position_Systems Administrator,Position_Systems Analyst,Position_Web Developer
0,1,4,0,0,1,0,0,0,0,0,0,0,0
1,2,6,1,0,1,0,0,0,0,0,0,0,0
2,3,17,1,0,0,0,0,0,0,0,0,0,1
3,4,7,1,0,0,0,0,0,0,0,1,0,0
4,5,13,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,396,19,0,0,0,0,0,0,0,0,0,0,0
396,397,20,0,0,0,0,0,0,0,0,0,0,1
397,398,9,0,0,0,0,0,0,1,0,0,0,0
398,399,18,1,1,0,0,0,0,0,0,0,0,0


In [110]:
import numpy as np
new_data=X.sample(5)
new_data_scaled = scaler.transform(new_data)

# Make predictions using the model
predictions = model.predict(new_data_scaled)


#reversing transformations
original_data = scaler.inverse_transform(new_data_scaled)
original_ID=original_data[:,:1].astype(int)


# Convert array to DataFrame with a column name
df = pd.DataFrame({
    'ID': original_ID.flatten(),  # Flatten to match the (5,) shape
    'Predicted_Salary': predictions
})


# Print DataFrame
print(df)

    ID  Predicted_Salary
0   44      86692.060609
1  364     162861.873090
2  279      79828.297645
3  178     142414.724040
4    9     138600.488973


In [111]:
import joblib

# Save the model to a file
model_filename = 'model.joblib'
joblib.dump(model, model_filename)

print(f"Model saved as {model_filename}")


Model saved as model.joblib


In [112]:
boto3_sm = boto3.client("sagemaker", region_name="ap-south-1")
session = sagemaker.Session()
region = session.boto_session.region_name
bucket = "mlops-salary-predictor-app"
print("Using Bucket:", bucket)


Using Bucket: mlops-salary-predictor-app


In [113]:
# Sagemaker takes training data from S3 bucket, so uploading data to S3 bucket
X_train_path = session.upload_data(path="X_train_df.csv", bucket=bucket)
X_test_path = session.upload_data(path="X_test_df.csv", bucket=bucket)
Y_train_path = session.upload_data(path="Y_train_df.csv", bucket=bucket)
Y_test_path = session.upload_data(path="Y_test_df.csv", bucket=bucket)

In [114]:
train_df=pd.read_csv("X_train_df.csv")

In [115]:
features = list(train_df.columns)
print(features)
label = features.pop(-1)
print(label)

['ID', 'Experience', 'Gender_M', 'Position_Database Administrator (DBA)', 'Position_DevOps Engineer', 'Position_IT Manager', 'Position_IT Security Analyst', 'Position_IT Support Specialist', 'Position_Network Administrator', 'Position_Software Engineer', 'Position_Systems Administrator', 'Position_Systems Analyst', 'Position_Web Developer', 'Salary']
Salary


0       69668
1      123370
2      188681
3      242819
4       71211
        ...  
275    110321
276    135066
277    115263
278    115769
279     90049
Name: Salary, Length: 280, dtype: int64

In [74]:


# Specify your S3 bucket and model file
model_file_path = 'model.joblib'
s3_model_path = 'data/model.joblib'  # The path where you want to store the model in S3

# Initialize the S3 client
s3 = boto3.client('s3')

# Upload the model to S3
s3.upload_file(model_file_path, bucket, s3_model_path)

print(f"Model uploaded to s3://{bucket}/{s3_model_path}")


Model uploaded to s3://mlops-salary-predictor-app/data/model.joblib


In [75]:
print(X_train)

[[-0.41085418 -0.32005212  0.98581488 ... -0.31994094 -0.29915575
  -0.34641016]
 [-0.84023834  0.65401954  0.98581488 ... -0.31994094 -0.29915575
  -0.34641016]
 [-1.66322463  0.97871009 -1.01438923 ... -0.31994094 -0.29915575
  -0.34641016]
 ...
 [ 0.59998768 -0.48239739 -1.01438923 ... -0.31994094 -0.29915575
  -0.34641016]
 [ 1.29773693 -0.15770684  0.98581488 ...  3.12557687 -0.29915575
  -0.34641016]
 [-0.90285686 -1.1317785  -1.01438923 ... -0.31994094 -0.29915575
  -0.34641016]]


In [119]:
%%writefile script.py

import argparse
import os
import json
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
import joblib
import pathlib
from io import StringIO
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import sagemaker
import boto3

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ =='__main__':

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # parser.add_argument('--n_estimators', type=int, default=100)
    # parser.add_argument('--random_state', type=int, default=0)
    # parser.add_argument('--epochs', type=int, default=10)
    # parser.add_argument('--batch-size', type=int, default=100)
    # parser.add_argument('--learning-rate', type=float, default=0.1)

    # an alternative way to load hyperparameters via SM_HPS environment variable.
    # parser.add_argument('--sm-hps', type=json.loads, default=os.environ['SM_HPS'])

    # input data and model directories
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
    parser.add_argument('--train_file', type=str, default='X_train_df.csv')
    parser.add_argument('--test_file', type=str, default='X_test_df.csv')

    args, _ = parser.parse_known_args()

    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = X_train_df[features]
    X_test = X_test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (80%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (20%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
    print("Training RandomForest Model.....")
    print()
    model = LinearRegression()
    model.fit(X_train, y_train)
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = mean_absolute_error(y_test, y_pred_test)


    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Absolute Error  is: ', test_acc)
 
   

Writing script.py


In [22]:

# categorical = ['Gender', 'Position']
# numerical = ['Experience (Years)']
# train_dicts = data[categorical + numerical].to_dict(orient='records')

# dv = DictVectorizer()
# X_train = dv.fit_transform(train_dicts)

# num_columns = X_train.shape[1]
# print("Number of columns in the feature matrix:", num_columns)

# target = 'Salary'
# Y_train = data[target].testues

# lr = LinearRegression()
# lr.fit(X_train, Y_train)

# y_pred = lr.predict(X_train)

# mean_squared_error(Y_train, y_pred, squared=False)

In [23]:
# def pred_salaries(inputfile):
#     df_sample=pd.read_csv(inputfile)
#     sample_dicts = df_sample[categorical + numerical].to_dict(orient='records')
#     X_sample = dv.transform(sample_dicts)
#     pred_salaries = lr.predict(X_sample)
    
#     # Create a DataFrame with ID and predicted salaries
#     result_df = pd.DataFrame({
#         'ID': df_sample["ID"],
#         'Predicted_Salary': pred_salaries
#     })
    
#     return result_df


    

In [24]:
# pred_salaries("./employee_data.csv")

In [25]:
# ### Create a Pickle file using serialization
# import joblib

# # Assuming `classifier` is your trained classifier object

# # Save the classifier using joblib
# joblib.dump(lr, "regressor.pkl")

testidate model

In [26]:

# df_test = pd.read_csv('./test_employee_data.csv')
# test_dicts = df_test[categorical + numerical].to_dict(orient='records')

# X_test = dv.transform(test_dicts)

# Y_test = df_test[target].testues

# y_pred = lr.predict(X_test)

# mean_squared_error(Y_test, y_pred, squared=False)


In [27]:
# import pickle

# # Save the model
# with open('model.pkl', 'wb') as f:
#     pickle.dump(lr, f)


In [28]:
# boto3_sm = boto3.client("sagemaker", region_name="ap-south-1")
# session = sagemaker.Session()
# region = session.boto_session.region_name
# bucket = "mlops-salary-predictor-app"
# print("Using Bucket:", bucket)

In [30]:
# %%writefile script.py

# import argparse
# import os
# import json
# import sklearn
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
# import joblib
# import pathlib
# from io import StringIO
# import boto3
# import pandas as pd
# import numpy as np

# def model_fn(model_dir):
#     clf = joblib.load(os.path.join(model_dir, "model.joblib"))
#     return clf

# if __name__ =='__main__':

#     print("[INFO] Extracting arguments")
#     parser = argparse.ArgumentParser()

#     # hyperparameters sent by the client are passed as command-line arguments to the script.
#     parser.add_argument('--n_estimators', type=int, default=100)
#     parser.add_argument('--random_state', type=int, default=0)
#     # parser.add_argument('--epochs', type=int, default=10)
#     # parser.add_argument('--batch-size', type=int, default=100)
#     # parser.add_argument('--learning-rate', type=float, default=0.1)

#     # an alternative way to load hyperparameters via SM_HPS environment variable.
#     # parser.add_argument('--sm-hps', type=json.loads, default=os.environ['SM_HPS'])

#     # input data and model directories
#     parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
#     parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
#     parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
#     parser.add_argument('--train_file', type=str, default='train_employee_data.csv')
#     parser.add_argument('--test_file', type=str, default='test_employee_data.csv')

#     args, _ = parser.parse_known_args()

#     print("SKLearn Version: ", sklearn.__version__)
#     print("Joblib Version: ", joblib.__version__)

#     print("[INFO] Reading data")
#     print()
#     train_df = pd.read_csv(os.path.join(args.train, args.train_file))
#     test_df = pd.read_csv(os.path.join(args.test, args.test_file))

#     features = list(train_df.columns)
#     # label = features.pop(-1)
    
#     print("Building training and testing datasets")
#     print()
#     X_train = train_df[features]
#     X_test = test_df[features]
#     Y_train = train_df[label]
#     Y_test = test_df[label]

#     print('Column order: ')
#     print(features)
#     print()
    
#     print("Label column is: ",label)
#     print()
    
#     print("Data Shape: ")
#     print()
#     print("---- SHAPE OF TRAINING DATA (70%) ----")
#     print(X_train.shape)
#     print(Y_train.shape)
#     print()
#     print("---- SHAPE OF TESTING DATA (30%) ----")
#     print(X_test.shape)
#     print(Y_test.shape)
#     print()
    
#     print("Training RandomForest Model.....")
#     print()
#     model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=3, n_jobs=None)
#     model.fit(X_train, Y_train)
#     print()

#     model_path = os.path.join(args.model_dir, "model.joblib")
#     joblib.dump(model, model_path)
#     print("Model persisted at " + model_path)
#     print()

    
#     y_pred_test = model.predict(X_test)
#     test_acc = accuracy_score(Y_test,y_pred_test)
#     test_rep = classification_report(Y_test,y_pred_test)

#     print()
#     print("---- METRICS RESULTS FOR TESTING DATA ----")
#     print()
#     print("Total Rows are: ", X_test.shape[0])
#     print('[TESTING] Model Accuracy is: ', test_acc)
#     print('[TESTING] Testing Report: ')
#     print(test_rep)